In [1]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar
from bokeh.palettes import Category10

output_notebook()

def plot_unit_gaussian_samples(D) :
    p = figure(plot_width=800, plot_height = 500, title = 'Sample from a unit {}D Gaussian'.format(D))
    xs = np.linspace(0,1,D)
    for color in Category10[10] : 
        ys = np.random.multivariate_normal(np.zeros(D),np.eye(D))
        p.line(xs, ys, line_width=1, color = color)

    return p


show(plot_unit_gaussian_samples(2))



Loading BokehJS ...

In [2]:
show(plot_unit_gaussian_samples(20))

In [3]:
def k(xs, ys, sigma=1, l=1) :
    #pairwise difference matrix
    dx = np.expand_dims(xs, 1) - np.expand_dims(ys,0)
    return (sigma**2)*np.exp(-((dx/l)**2)/2)

def m(x) : 
    return np.zeros_like(x)

In [4]:
N = 100
x = np.linspace(-2,2,N)
y = np.linspace(-2,2,N)
d = k(x,y)

color_mapper = LinearColorMapper(palette="Plasma256",low=0, high=1)

p = figure(plot_width=400, plot_height=400, x_range=(-2,2), y_range=(-2,2),title='Visualisation of k(x,x\')',
          x_axis_label = 'x', y_axis_label='x\'',toolbar_location=None)

p.image(image=[d],color_mapper=color_mapper, x=-2, y=-2, dw=4, dh=4)

color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                    label_standoff=12,border_line_color=None,location=(0,0))

p.add_layout(color_bar,'right')

show(p)

In [20]:
p = figure(plot_width=800,plot_height=500)
D = 6
xs = np.linspace(0, 1, D)

print(np.expand_dims(xs,0))
print(np.expand_dims(xs,1))
print(np.expand_dims(xs,0) - np.expand_dims(xs,1))

print(xs)
print(k(xs,xs))

for color in Category10[10] :
    ys = np.random.multivariate_normal(m(xs), k(xs,xs))
    p.circle(xs, ys, size=3,color=color)
    p.line(xs, ys, line_width=1, color=color)
    
show(p)

[[ 0.   0.2  0.4  0.6  0.8  1. ]]
[[ 0. ]
 [ 0.2]
 [ 0.4]
 [ 0.6]
 [ 0.8]
 [ 1. ]]
[[ 0.   0.2  0.4  0.6  0.8  1. ]
 [-0.2  0.   0.2  0.4  0.6  0.8]
 [-0.4 -0.2  0.   0.2  0.4  0.6]
 [-0.6 -0.4 -0.2  0.   0.2  0.4]
 [-0.8 -0.6 -0.4 -0.2  0.   0.2]
 [-1.  -0.8 -0.6 -0.4 -0.2  0. ]]
[ 0.   0.2  0.4  0.6  0.8  1. ]
[[ 1.          0.98019867  0.92311635  0.83527021  0.72614904  0.60653066]
 [ 0.98019867  1.          0.98019867  0.92311635  0.83527021  0.72614904]
 [ 0.92311635  0.98019867  1.          0.98019867  0.92311635  0.83527021]
 [ 0.83527021  0.92311635  0.98019867  1.          0.98019867  0.92311635]
 [ 0.72614904  0.83527021  0.92311635  0.98019867  1.          0.98019867]
 [ 0.60653066  0.72614904  0.83527021  0.92311635  0.98019867  1.        ]]


In [29]:
n = 100
xs = np.linspace(-5, 5, n)
K = k(xs, xs)
mu = m(xs)

p = figure(plot_width=800, plot_height = 500)

for color in Category10[5] : 
    ys = np.random.multivariate_normal(mu, K)
    p.line(xs,ys, line_width=2, color=color)
    
show(p)

In [7]:
coefs = [6, -2.5, -2.4, -0.1, 0.2, 0.03]

def f(x) :
    total = 0
    for exp, coef in enumerate(coefs) :
        total  += coef * (x **exp)
    return total

xs = np.linspace(-5.0, 3.5, 100)
ys = f(xs)

p = figure(plot_width=800, plot_height=400, x_axis_label='x',
          y_axis_label='f(x)',title='the hidden function f(x)')
p.line(xs, ys, line_width=2)
show(p)



In [30]:
x_obs = np.array([-4, -1.5, 0, 1.5, 2.5, 2.7])
y_obs = f(x_obs)

x_s = np.linspace(-8, 7, 80)

K = k(x_obs, x_obs)
K_s = k(x_obs,x_s)
K_ss = k(x_s, x_s)

K_sTKinv = np.matmul(K_s.T, np.linalg.pinv(K))

mu_s = m(x_s) + np.matmul(K_sTKinv, y_obs - m(x_obs))
Sigma_s = K_ss - np.matmul(K_sTKinv, K_s)




In [35]:
p = figure(plot_width = 800, plot_height=600,y_range=(-7,8))

y_true = f(x_s)
p.line(x_s, y_true, line_width=3, color='black', alpha=0.4,line_dash='dashed',legend='True F(x)')

p.cross(x_obs, y_obs, size=20, legend='Training data')

stds = np.sqrt(Sigma_s.diagonal())
err_xs = np.concatenate((x_s, np.flip(x_s,0)))
err_ys = np.concatenate((mu_s + 2 * stds, np.flip(mu_s - 2 * stds, 0)))
p.patch(err_xs, err_ys, alpha=0.2, line_width=0, color='gray',legend='Uncertainty')

for color in Category10[3] :
    y_s = np.random.multivariate_normal(mu_s,Sigma_s)
    p.line(x_s, y_s, line_width=1, color=color)
    
p.line(x_s, mu_s, line_width=3, color='blue', alpha=0.4, legend='Mean')
show(p)





In [36]:
# bridg.land/posts/gaussian-processes-1